In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello world

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>パッケージバージョン</b></summary>

このページのコードは、以下の要件を使用して開発されました。
これらのバージョンまたはそれ以降のバージョンの使用を推奨します。

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
この例は2つの部分で構成されています。まず、シンプルな量子プログラムを作成し、量子処理ユニット(QPU)で実行します。実際の量子研究にははるかに堅牢なプログラムが必要なため、第2セクション([多数の量子ビットへのスケール](#scale-to-large-numbers-of-qubits))では、シンプルなプログラムをユーティリティレベルまでスケールアップします。
## インストールと認証
1. まだQiskitをインストールしていない場合は、[クイックスタート](/guides/quick-start)ガイドで手順を確認してください。

    - 量子ハードウェアでジョブを実行するためにQiskit Runtimeをインストールします:

        ```bash
        pip install qiskit-ibm-runtime
        ```

    - Jupyterノートブックをローカルで実行するための環境を設定します:

        ```bash
        pip install jupyter
        ```

2. 無料の[Openプラン](/guides/plans-overview#open-plan)を通じて量子ハードウェアへのアクセスのための認証を設定します。

    (アカウントへの招待メールを受け取った場合は、代わりに[招待されたユーザーの手順](/guides/cloud-setup-invited)に従ってください。)

    - [IBM Quantum Platform](https://quantum.cloud.ibm.com/)にアクセスして、ログインまたはアカウントを作成します。
         > **Note:** プロキシサーバー経由で接続する場合は、Qiskit Runtime v0.44.0以降を使用する必要があります。
    - [ダッシュボード](https://quantum.cloud.ibm.com/)でAPIキー(*APIトークン*とも呼ばれます)を生成し、安全な場所にコピーします。
    - [インスタンス](https://quantum.cloud.ibm.com/instances)ページにアクセスし、使用したいインスタンスを見つけます。そのCRNにカーソルを合わせてクリックしてコピーします。

    - 次のコードで認証情報をローカルに保存します:

        ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        QiskitRuntimeService.save_account(
        token="<your-api-key>", # IBM Quantum Platformホームダッシュボードから作成して保存した44文字のAPI_KEYを使用
        instance="<CRN>", # オプション
        )
        ```

3. これで、Qiskit Runtime Serviceに認証が必要なときはいつでも、このPythonコードを使用できます:
    ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        # サービスが必要なときに毎回実行
        service = QiskitRuntimeService()
    ```
> **Info:** 公共のコンピューターやその他の安全でない環境を使用している場合は、認証情報を安全に保つために、代わりに[手動認証手順](/guides/cloud-setup-untrusted)に従ってください。
## シンプルな量子プログラムを作成して実行する
Qiskitパターンを使用して量子プログラムを書くための4つのステップは以下の通りです:

1.  問題を量子ネイティブ形式にマッピングします。

2.  回路と演算子を最適化します。

3.  量子プリミティブ関数を使用して実行します。

4.  結果を分析します。

### ステップ1. 問題を量子ネイティブ形式にマッピングする
量子プログラムでは、*量子回路*は量子命令を表すネイティブ形式であり、*演算子*は測定される観測量を表します。回路を作成する際、通常は新しい[`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class)オブジェクトを作成し、順番に命令を追加します。
次のコードセルは、*ベル状態*を生成する回路を作成します。ベル状態は、2つの量子ビットが完全にもつれ合った状態です。

> **Note:** Qiskit SDKは、$n^{th}$桁が値$1 \ll n$または$2^n$を持つLSb 0ビット番号付けを使用します。詳細については、[Qiskit SDKでのビット順序](/guides/bit-ordering)のトピックを参照してください。

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from matplotlib import pyplot as plt
# Uncomment the next line if you want to use a simulator:
# from qiskit_ibm_runtime.fake_provider import FakeBelemV2


# Create a new circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)

# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

<Image src="../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg)

利用可能なすべての操作については、ドキュメントの[`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class)を参照してください。
量子回路を作成する際には、実行後に返されるデータの種類も考慮する必要があります。Qiskitは2つの方法でデータを返します: 測定する量子ビットのセットの確率分布を取得するか、観測量の期待値を取得できます。[Qiskitプリミティブ](/guides/get-started-with-primitives)(詳細は[ステップ3](#step-3-execute-using-the-quantum-primitives)で説明)を使用して、これら2つの方法のいずれかで回路を測定するようにワークロードを準備します。

この例では、`qiskit.quantum_info`サブモジュールを使用して期待値を測定します。これは演算子(量子状態を変更するアクションまたはプロセスを表すために使用される数学的オブジェクト)を使用して指定されます。次のコードセルは、6つの2量子ビットパウリ演算子を作成します: `IZ`、`IX`、`ZI`、`XI`、`ZZ`、`XX`。

In [2]:
# Set up six different observables.

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

> **Note:** ここで、`ZZ`演算子のようなものは、テンソル積$Z\otimes Z$の省略形であり、量子ビット1のZと量子ビット0のZを一緒に測定し、量子ビット1と量子ビット0の間の相関に関する情報を取得することを意味します。このような期待値は、通常$\langle Z_1 Z_0 \rangle$とも書かれます。
> 
> 状態がもつれている場合、$\langle Z_1 Z_0 \rangle$の測定は$\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$の測定とは異なるはずです。上記で説明した回路によって作成される特定のもつれ状態の場合、$\langle Z_1 Z_0 \rangle$の測定は1になり、$\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$の測定はゼロになるはずです。
<span id="optimize"></span>

### ステップ2. 回路と演算子を最適化する

デバイスで回路を実行する場合、回路に含まれる命令のセットを最適化し、回路の全体的な深さ(おおよそ命令の数)を最小限に抑えることが重要です。これにより、エラーとノイズの影響を軽減し、可能な限り最良の結果を得ることができます。さらに、回路の命令はバックエンドデバイスの[命令セットアーキテクチャ(ISA)](/guides/transpile#instruction-set-architecture)に適合し、デバイスの基底ゲートと量子ビット接続性を考慮する必要があります。

次のコードは、ジョブを送信する実際のデバイスをインスタンス化し、そのバックエンドのISAに一致するように回路と観測量を変換します。これには、すでに[認証情報を保存](/guides/cloud-setup)している必要があります。

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg)

### ステップ3. 量子プリミティブを使用して実行する
量子コンピューターはランダムな結果を生成する可能性があるため、通常、回路を何度も実行して出力のサンプルを収集します。`Estimator`クラスを使用して観測量の値を推定できます。`Estimator`は2つの[プリミティブ](/guides/get-started-with-primitives)の1つで、もう1つは量子コンピューターからデータを取得するために使用できる`Sampler`です。これらのオブジェクトには、[プリミティブ統一ブロック(PUB)](/guides/primitives#sampler)を使用して、回路、観測量、パラメータ(該当する場合)の選択を実行する`run()`メソッドがあります。

In [4]:
# Construct the Estimator instance.

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d5k96q4jt3vs73ds5tgg


After a job is submitted, you can wait until either the job is completed within your current python instance, or use the `job_id` to retrieve the data at a later time.  (See the [section on retrieving jobs](/docs/guides/monitor-job#retrieve-job-results-at-a-later-time) for details.)

After the job completes, examine its output through the job's `result()` attribute.

In [5]:
# This is the result of the entire submission.  You submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had six observables,
# so contains information on all six.
pub_result = job.result()[0]

In [6]:
# Check there are six observables.
# If not, edit the comments in the previous cell and update this test.
assert len(pub_result.data.evs) == 6

<Admonition type="note" title="Alternative: run the example using a simulator">
  When you run your quantum program on a real device, your workload must wait in a queue before it runs. To save time, you can instead use the following code to run this small workload on the [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) with the Qiskit Runtime local testing mode. Note that this is only possible for a small circuit. When you scale up in the next section, you will need to use a real device.

  ```python

  # Use the following code instead if you want to run on a simulator:

  from qiskit_ibm_runtime.fake_provider import FakeBelemV2
  backend = FakeBelemV2()
  estimator = Estimator(backend)

  # Convert to an ISA circuit and layout-mapped observables.

  pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
  isa_circuit = pm.run(qc)
  mapped_observables = [
      observable.apply_layout(isa_circuit.layout) for observable in observables
  ]

  job = estimator.run([(isa_circuit, mapped_observables)])
  result = job.result()

  # This is the result of the entire submission.  You submitted one Pub,
  # so this contains one inner result (and some metadata of its own).

  job_result = job.result()

  # This is the result from our single pub, which had five observables,
  # so contains information on all five.

  pub_result = job.result()[0]
  ```
</Admonition>

### Step 4. Analyze the results

The analyze step is typically where you might post-process your results using, for example, measurement error mitigation or zero noise extrapolation (ZNE). You might feed these results into another workflow for further analysis or prepare a plot of the key values and data. In general, this step is specific to your problem.  For this example, plot each of the expectation values that were measured for our circuit.

The expectation values and standard deviations for the observables you specified to Estimator are accessed through the job result's `PubResult.data.evs` and `PubResult.data.stds` attributes. To obtain the results from Sampler, use the `PubResult.data.meas.get_counts()` function, which will return a `dict` of measurements in the form of bitstrings as keys and counts as their corresponding values. For more information, see [Get started with Sampler.](/docs/guides/get-started-with-primitives#get-started-with-sampler)

In [ ]:
# Plot the result

values = pub_result.data.evs

errors = pub_result.data.stds

# plotting graph
plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg" alt="Output of the previous code cell" />

> **Note:** 量子プログラムを実際のデバイスで実行する場合、ワークロードは実行前にキューで待機する必要があります。時間を節約するために、代わりに次のコードを使用して、Qiskit Runtimeローカルテストモードで[`fake_provider`](../api/qiskit-ibm-runtime/fake-provider)でこの小さなワークロードを実行できます。これは小さな回路でのみ可能であることに注意してください。次のセクションでスケールアップする場合、実際のデバイスを使用する必要があります。
> 
>   ```python
> 
>   # シミュレーターで実行したい場合は、代わりに次のコードを使用してください:
> 
>   from qiskit_ibm_runtime.fake_provider import FakeBelemV2
>   backend = FakeBelemV2()
>   estimator = Estimator(backend)
> 
>   # ISA回路とレイアウトマッピングされた観測量に変換します。
> 
>   pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
>   isa_circuit = pm.run(qc)
>   mapped_observables = [
>       observable.apply_layout(isa_circuit.layout) for observable in observables
>   ]
> 
>   job = estimator.run([(isa_circuit, mapped_observables)])
>   result = job.result()
> 
>   # これは送信全体の結果です。1つのPubを送信したので、
>   # 1つの内部結果(およびそれ自体のメタデータ)が含まれています。
> 
>   job_result = job.result()
> 
>   # これは単一のpubからの結果で、5つの観測量があったため、
>   # すべての5つに関する情報が含まれています。
> 
>   pub_result = job.result()[0]
>   ```
### ステップ4. 結果を分析する
分析ステップは、通常、測定エラー軽減やゼロノイズ外挿(ZNE)などを使用して結果を後処理する場所です。これらの結果をさらなる分析のために別のワークフローにフィードしたり、主要な値とデータのプロットを準備したりすることもできます。一般に、このステップは問題に固有のものです。この例では、回路に対して測定された各期待値をプロットします。

Estimatorに指定した観測量の期待値と標準偏差は、ジョブ結果の`PubResult.data.evs`および`PubResult.data.stds`属性を通じてアクセスされます。Samplerから結果を取得するには、`PubResult.data.meas.get_counts()`関数を使用します。これは、ビット文字列をキーとし、カウントを対応する値として測定の`dict`を返します。詳細については、[Samplerを使ってみる](/guides/get-started-with-primitives#get-started-with-sampler)を参照してください。

In [8]:
# Make sure the results follow the claim from the previous markdown cell.
# This can happen when the device occasionally behaves strangely. If this cell
# fails, you may just need to run the notebook again.

_results = {obs: val for obs, val in zip(observables_labels, values)}
for _label in ["IZ", "IX", "ZI", "XI"]:
    assert abs(_results[_label]) < 0.2
for _label in ["XX", "ZZ"]:
    assert _results[_label] > 0.8

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg)

量子ビット0と1について、XとZの両方の独立した期待値は0であり、相関(`XX`と`ZZ`)は1であることに注意してください。これは量子もつれの特徴です。

In [ ]:
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.

    Args:
        n (int): Number of qubits in the n-qubit GHZ state

    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n - 1):
            qc.cx(i, i + 1)
    else:
        raise Exception("n is not a valid input")
    return qc


# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
n = 100
qc = get_qc_for_n_qubit_GHZ_state(n)

## 多数の量子ビットへのスケール
量子コンピューティングにおいて、ユーティリティスケールの作業は、この分野で進歩を遂げるために重要です。このような作業には、はるかに大規模な計算が必要です。100個を超える量子ビットと1000個を超えるゲートを使用する回路での作業が必要です。この例では、100量子ビットのGHZ状態を作成および分析することで、IBM&reg; QPUでユーティリティスケールの作業を実現する方法を示します。Qiskitパターンワークフローを使用し、各量子ビットに対して期待値$\langle Z_0 Z_i \rangle $を測定して終了します。

### ステップ1. 問題をマッピングする
$n$量子ビットのGHZ状態(基本的に拡張ベル状態)を準備する`QuantumCircuit`を返す関数を書き、その関数を使用して100量子ビットのGHZ状態を準備し、測定する観測量を収集します。

In [ ]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (n - 2 - i) for i in range(n - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

次に、関心のある演算子にマッピングします。この例では、量子ビット間の`ZZ`演算子を使用して、距離が離れるにつれての振る舞いを調べます。遠くの量子ビット間の期待値がますます不正確(破損)になることで、存在するノイズのレベルが明らかになります。

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(
    simulator=False, operational=True, min_num_qubits=100
)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

isa_circuit = pm.run(qc)
isa_operators_list = [op.apply_layout(isa_circuit.layout) for op in operators]

### Step 3. Execute on hardware

Submit the job and enable error suppression by using a technique to reduce errors called [dynamical decoupling.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) The resilience level specifies how much resilience to build against errors. Higher levels generate more accurate results, at the expense of longer processing times.  For further explanation of the options set in the following code, see [Configure error mitigation for Qiskit Runtime.](/docs/guides/configure-error-mitigation)

In [ ]:
options = EstimatorOptions()
options.resilience_level = 1
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [13]:
# Submit the circuit to Estimator
job = estimator.run([(isa_circuit, isa_operators_list)])
job_id = job.job_id()
print(job_id)

d5k9mmqvcahs73a1ni3g


### ステップ3. ハードウェアで実行する
ジョブを送信し、エラーを減らす技術である[動的デカップリング](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options)を使用してエラー抑制を有効にします。レジリエンスレベルは、エラーに対してどれだけのレジリエンスを構築するかを指定します。レベルが高いほど、処理時間が長くなる代わりに、より正確な結果が生成されます。次のコードで設定されているオプションの詳細な説明については、[Qiskit Runtimeのエラー軽減の設定](/guides/configure-error-mitigation)を参照してください。

In [ ]:
# data
data = list(range(1, len(operators) + 1))  # Distance between the Z operators
result = job.result()[0]
values = result.data.evs  # Expectation value at each Z operator.
values = [
    v / values[0] for v in values
]  # Normalize the expectation values to evaluate how they decay with distance.

# plotting graph
plt.plot(data, values, marker="o", label="100-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/de91ebd0-0.svg" alt="Output of the previous code cell" />

The previous plot shows that as the distance between qubits increases, the signal decays because of the presence of noise.

## Next steps

<Admonition type="tip" title="Recommendations">
  -   Try one of these tutorials:
      - [Ground-state energy estimation of the Heisenberg chain with VQE](/docs/tutorials/spin-chain-vqe)
      - Solve optimization problems using [QAOA](/docs/tutorials/quantum-approximate-optimization-algorithm)
      - Train [quantum kernel](/docs/tutorials/quantum-kernel-training) models for machine learning tasks
  - Find detailed installation instructions in the [Install Qiskit](/docs/guides/install-qiskit) guide.
  - If you prefer not to install Qiskit locally, read about options to use Qiskit in an [online development environment.](/docs/guides/online-lab-environments)
  - To save multiple account credentials or to specify other account options, see detailed instructions in the [Save your login credentials](/docs/guides/save-credentials#save-your-access-credentials) guide.
</Admonition>